# 서울특별시 공공자전거 실시간 대여정보

1.	rackTotCnt	거치대개수
2.	stationName	대여소이름
3.	parkingBikeTotCnt	자전거주차총건수
4.	shared	거치율
5.	stationLatitude	위도
6.	stationLongitude	경도
7.	stationId	대여소ID

In [19]:
import requests
import json
import folium
import os
from selenium import webdriver
import pandas as pd
import numpy as np

In [22]:
api_key = '75636a72567073753733697370467a'
startnum = 1 # 1001, 2001, 3001...
endnum = 1000 # 2000, 3000, 4000 ...

In [23]:
url = f"http://openapi.seoul.go.kr:8088/{api_key}/json/bikeList/{startnum}/{endnum}/"
data = requests.get(url)
result = json.loads(data.text) 
print(json.dumps(result, indent="\t"))

{
	"rentBikeStatus": {
		"list_total_count": 1000,
		"RESULT": {
			"CODE": "INFO-000",
			"MESSAGE": "\uc815\uc0c1 \ucc98\ub9ac\ub418\uc5c8\uc2b5\ub2c8\ub2e4."
		},
		"row": [
			{
				"rackTotCnt": "22",
				"stationName": "102. \ub9dd\uc6d0\uc5ed 1\ubc88\ucd9c\uad6c \uc55e",
				"parkingBikeTotCnt": "19",
				"shared": "86",
				"stationLatitude": "37.55564880",
				"stationLongitude": "126.91062927",
				"stationId": "ST-4"
			},
			{
				"rackTotCnt": "16",
				"stationName": "103. \ub9dd\uc6d0\uc5ed 2\ubc88\ucd9c\uad6c \uc55e",
				"parkingBikeTotCnt": "18",
				"shared": "113",
				"stationLatitude": "37.55495071",
				"stationLongitude": "126.91083527",
				"stationId": "ST-5"
			},
			{
				"rackTotCnt": "15",
				"stationName": "104. \ud569\uc815\uc5ed 1\ubc88\ucd9c\uad6c \uc55e",
				"parkingBikeTotCnt": "0",
				"shared": "0",
				"stationLatitude": "37.55062866",
				"stationLongitude": "126.91498566",
				"stationId": "ST-6"
			},
			{
				"rackTotCnt": "7",
				"stationN

In [24]:
rows = result["rentBikeStatus"]["row"]
rows

[{'rackTotCnt': '22',
  'stationName': '102. 망원역 1번출구 앞',
  'parkingBikeTotCnt': '19',
  'shared': '86',
  'stationLatitude': '37.55564880',
  'stationLongitude': '126.91062927',
  'stationId': 'ST-4'},
 {'rackTotCnt': '16',
  'stationName': '103. 망원역 2번출구 앞',
  'parkingBikeTotCnt': '18',
  'shared': '113',
  'stationLatitude': '37.55495071',
  'stationLongitude': '126.91083527',
  'stationId': 'ST-5'},
 {'rackTotCnt': '15',
  'stationName': '104. 합정역 1번출구 앞',
  'parkingBikeTotCnt': '0',
  'shared': '0',
  'stationLatitude': '37.55062866',
  'stationLongitude': '126.91498566',
  'stationId': 'ST-6'},
 {'rackTotCnt': '7',
  'stationName': '105. 합정역 5번출구 앞',
  'parkingBikeTotCnt': '1',
  'shared': '14',
  'stationLatitude': '37.55000687',
  'stationLongitude': '126.91482544',
  'stationId': 'ST-7'},
 {'rackTotCnt': '14',
  'stationName': '106. 합정역 7번출구 앞',
  'parkingBikeTotCnt': '5',
  'shared': '36',
  'stationLatitude': '37.54864502',
  'stationLongitude': '126.91282654',
  'stationId'

In [28]:
lat_sum = 0
lon_sum = 0
for i in rows:
    lat = float(i["stationLatitude"]) 
    lon = float(i["stationLongitude"])
    lat_sum += lat
    lon_sum += lon
lat_avr = lat_sum/len(rows)
lon_avr = lon_sum/len(rows)

map = folium.Map(location=[lat_avr, lon_avr], zoom_start=14)
for i in rows:
    bike_num = int(i["parkingBikeTotCnt"])
    station_name = i["stationName"]
    lat = i["stationLatitude"]
    lon = i["stationLongitude"]
    if bike_num < 3:
        color = "red"
    elif 3 <= bike_num < 7:
        color = "blue"
    elif 7 <= bike_num:
        color = "green"
    folium.Marker(location=[lat, lon], popup=station_name, tooltip=bike_num, icon=folium.Icon(color=color)).add_to(map)
map.save("./map.html")

file_path = os.path.abspath("./map.html")
browser = webdriver.Chrome(r"C:\Users\psuny\Desktop\chromedriver.exe")
browser.get(file_path) 

WebDriverException: Message: 'chromedriver.exe' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home


In [25]:
data = pd.DataFrame(rows)
data.head()

,rackTotCnt,stationName,parkingBikeTotCnt,shared,stationLatitude,stationLongitude,stationId
0,22,102. 망원역 1번출구 앞,19,86,37.55564880,126.91062927,ST-4
1,16,103. 망원역 2번출구 앞,18,113,37.55495071,126.91083527,ST-5
2,15,104. 합정역 1번출구 앞,0,0,37.55062866,126.91498566,ST-6
3,7,105. 합정역 5번출구 앞,1,14,37.55000687,126.91482544,ST-7
4,14,106. 합정역 7번출구 앞,5,36,37.54864502,126.91282654,ST-8


- 이후 더 startnum, endnum 바꿔서 한 뒤
- df = data.append(data1)